In [ ]:
## This file implements neural networks before and after lasso selection for p0040kpresabs_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for each model.

In [1]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p0040kpresabs_qual.csv')
df.shape

(253, 182)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      2
1      0
2      2
3      2
4      2
      ..
248    2
249    1
250    1
251    2
252    2
Name: pheno, Length: 253, dtype: int64

In [5]:
df.head()

,id,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTACAAAAGAGGAAGCAGATAAACTTTATCAACCTATCGGTTCTTCGCAGCCGTCACTGAATATTTGGACAGGCAGTGAAACAGAATATAATTATTTG,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTT,TTTGGACAGGCAGTGAAAC,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,TTTCGCTGTTGCAACATCTTTAACAGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACT,...,AGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTT,AGCAGATAAACTT,AGCAGATAAACTTT,ACTTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTT,ACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTGATTGT,ACATCTTTAACAGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTT,ACAAGTCGCTGAAATATT,AACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTT,AACTGCTCATACAGTTAAAACAGCACAAACTGCTCATACAGTTAAAACAGCACAAACTGCTCAAGAACAAAATAAAGTTCAAACACCTGTTAAAGATGTT,pheno
0,107,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,2
1,109,1,0,1,0,0,0,1,0,0,...,0,1,1,1,0,0,1,0,0,0
2,115,0,0,1,0,0,0,1,0,0,...,0,1,1,0,0,0,1,0,0,2
3,120335,0,1,1,1,1,1,1,1,1,...,1,1,1,0,1,1,1,1,1,2
4,120337,0,1,1,1,1,1,1,1,1,...,1,1,1,0,1,1,1,1,1,2


In [6]:
df['pheno'].value_counts()

2    95
1    94
0    64
Name: pheno, dtype: int64

In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(253, 181)

In [9]:
df_clean.head()

,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTACAAAAGAGGAAGCAGATAAACTTTATCAACCTATCGGTTCTTCGCAGCCGTCACTGAATATTTGGACAGGCAGTGAAACAGAATATAATTATTTG,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTT,TTTGGACAGGCAGTGAAAC,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,TTTCGCTGTTGCAACATCTTTAACAGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACT,TTTCAGCGACTT,...,AGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTT,AGCAGATAAACTT,AGCAGATAAACTTT,ACTTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTT,ACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTGATTGT,ACATCTTTAACAGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTT,ACAAGTCGCTGAAATATT,AACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTT,AACTGCTCATACAGTTAAAACAGCACAAACTGCTCATACAGTTAAAACAGCACAAACTGCTCAAGAACAAAATAAAGTTCAAACACCTGTTAAAGATGTT,pheno
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,2
1,1,0,1,0,0,0,1,0,0,1,...,0,1,1,1,0,0,1,0,0,0
2,0,0,1,0,0,0,1,0,0,1,...,0,1,1,0,0,0,1,0,0,2
3,0,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,1,1,1,1,2
4,0,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,1,1,1,1,2


In [10]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 181) (253,)


In [11]:
############# Fully-Connected Neural Network ################

In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

Using TensorFlow backend.


In [13]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y)

In [14]:
dat = pd.DataFrame(X_test.iloc[:,0])
dat['test'] = y_test

In [15]:
dat

,id,test
221,NRS266,2
177,NRS209,1
223,NRS272,2
49,CFBREBSa114,2
53,CFBREBSa119,1
...,...,...
162,NRS187,1
237,SR1287,0
84,CFBRSa50,0
168,NRS196,2


In [16]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [17]:
#### neural network on over-sampling data
model1 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [18]:
model1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model1.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 631us/step - loss: 1.1848 - accuracy: 0.3672 - val_loss: 1.0511 - val_accuracy: 0.5000
Epoch 2/100
177/177 [==============================] - 0s 92us/step - loss: 1.1064 - accuracy: 0.3559 - val_loss: 1.0558 - val_accuracy: 0.4605
Epoch 3/100
177/177 [==============================] - 0s 128us/step - loss: 1.0767 - accuracy: 0.3955 - val_loss: 1.0244 - val_accuracy: 0.4605
Epoch 4/100
177/177 [==============================] - 0s 135us/step - loss: 1.0554 - accuracy: 0.4407 - val_loss: 1.0151 - val_accuracy: 0.4605
Epoch 5/100
177/177 [==============================] - 0s 122us/step - loss: 1.0423 - accuracy: 0.4633 - val_loss: 1.0166 - val_accuracy: 0.5000
Epoch 6/100
177/177 [==============================] - 0s 126us/step - loss: 1.0294 - accuracy: 0.4633 - val_loss: 1.0007 - val_accuracy: 0.5000
Epoch 7/100
177/177 [==============================] - 0s 112us/step - loss: 1.0219 - 

Epoch 57/100
177/177 [==============================] - 0s 110us/step - loss: 0.8382 - accuracy: 0.6215 - val_loss: 0.9384 - val_accuracy: 0.5395
Epoch 58/100
177/177 [==============================] - 0s 115us/step - loss: 0.8357 - accuracy: 0.6271 - val_loss: 0.9222 - val_accuracy: 0.5789
Epoch 59/100
177/177 [==============================] - 0s 92us/step - loss: 0.8349 - accuracy: 0.6158 - val_loss: 0.9090 - val_accuracy: 0.5526
Epoch 60/100
177/177 [==============================] - 0s 105us/step - loss: 0.8342 - accuracy: 0.6271 - val_loss: 0.9055 - val_accuracy: 0.5395
Epoch 61/100
177/177 [==============================] - 0s 110us/step - loss: 0.8334 - accuracy: 0.6441 - val_loss: 0.9188 - val_accuracy: 0.5395
Epoch 62/100
177/177 [==============================] - 0s 112us/step - loss: 0.8270 - accuracy: 0.6441 - val_loss: 0.9221 - val_accuracy: 0.5395
Epoch 63/100
177/177 [==============================] - 0s 108us/step - loss: 0.8249 - accuracy: 0.6384 - val_loss: 0.9142 - 

In [52]:
acc_test1 = model1.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test1*100))

76/76 [==============================] - 0s 201us/step
test accuracy: 51.32%


In [20]:
pred = model1.predict_classes(X_test)
pred

array([2, 1, 2, 2, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 1, 2, 0, 2, 2, 0, 0, 2,
       2, 0, 2, 0, 2, 2, 2, 2, 1, 0, 2, 0, 1, 0, 0, 2, 0, 2, 2, 1, 1, 1,
       2, 0, 1, 2, 0, 2, 0, 1, 2, 2, 1, 0, 2, 2, 0, 2, 2, 2, 0, 1, 2, 0,
       2, 0, 1, 0, 0, 2, 0, 2, 2, 0])

In [21]:
dat['pred'] = pred
dat

,id,test,pred
221,NRS266,2,2
177,NRS209,1,1
223,NRS272,2,2
49,CFBREBSa114,2,2
53,CFBREBSa119,1,0
...,...,...,...
162,NRS187,1,2
237,SR1287,0,0
84,CFBRSa50,0,2
168,NRS196,2,2


In [24]:
proba1 = model1.predict_proba(X_test)
dat_proba1 = pd.DataFrame(proba1)

In [25]:
dat_proba1

,0,1,2
0,0.040479,0.280500,0.679021
1,0.176141,0.757298,0.066560
2,0.155533,0.366461,0.478006
3,0.123646,0.293973,0.582381
4,0.489394,0.219591,0.291015
...,...,...,...
71,0.240571,0.226299,0.533130
72,0.514265,0.181804,0.303931
73,0.376491,0.224424,0.399085
74,0.015289,0.341986,0.642725


In [26]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [27]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p40.csv", index = False,
         header=None)

In [56]:
hist1 = model1.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 146us/step - loss: 0.7289 - accuracy: 0.6893 - val_loss: 0.9653 - val_accuracy: 0.5132
Epoch 2/100
177/177 [==============================] - 0s 151us/step - loss: 0.7316 - accuracy: 0.6949 - val_loss: 0.9683 - val_accuracy: 0.5000
Epoch 3/100
177/177 [==============================] - 0s 165us/step - loss: 0.7284 - accuracy: 0.7006 - val_loss: 0.9663 - val_accuracy: 0.4868
Epoch 4/100
177/177 [==============================] - 0s 142us/step - loss: 0.7264 - accuracy: 0.7062 - val_loss: 0.9641 - val_accuracy: 0.5000
Epoch 5/100
177/177 [==============================] - 0s 169us/step - loss: 0.7228 - accuracy: 0.6949 - val_loss: 0.9605 - val_accuracy: 0.5132
Epoch 6/100
177/177 [==============================] - 0s 188us/step - loss: 0.7230 - accuracy: 0.6893 - val_loss: 0.9592 - val_accuracy: 0.5000
Epoch 7/100
177/177 [==============================] - 0s 133us/step - loss: 0.7226 -

Epoch 57/100
177/177 [==============================] - 0s 164us/step - loss: 0.6437 - accuracy: 0.7514 - val_loss: 1.0273 - val_accuracy: 0.5132
Epoch 58/100
177/177 [==============================] - 0s 186us/step - loss: 0.6380 - accuracy: 0.7571 - val_loss: 1.0317 - val_accuracy: 0.5000
Epoch 59/100
177/177 [==============================] - 0s 309us/step - loss: 0.6381 - accuracy: 0.7345 - val_loss: 1.0283 - val_accuracy: 0.5000
Epoch 60/100
177/177 [==============================] - 0s 182us/step - loss: 0.6349 - accuracy: 0.7401 - val_loss: 1.0312 - val_accuracy: 0.4868
Epoch 61/100
177/177 [==============================] - 0s 173us/step - loss: 0.6348 - accuracy: 0.7401 - val_loss: 1.0321 - val_accuracy: 0.4868
Epoch 62/100
177/177 [==============================] - 0s 193us/step - loss: 0.6309 - accuracy: 0.7458 - val_loss: 1.0329 - val_accuracy: 0.5395
Epoch 63/100
177/177 [==============================] - 0s 171us/step - loss: 0.6348 - accuracy: 0.7458 - val_loss: 1.0270 -

In [57]:
print('train accuracy: %.2f%%' % (np.mean(hist1.history['accuracy'])*100))

train accuracy: 73.18%


In [17]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [18]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS027,1,0,0.759813,0.001304,0.238883
1,p0017kpresabs_qual,NRS246,1,0,0.947819,0.009591,0.042591
2,p0017kpresabs_qual,NRS218,2,1,0.008693,0.989390,0.001916
3,p0017kpresabs_qual,CFBRSa70,2,0,0.813774,0.001482,0.184744
4,p0017kpresabs_qual,NRS177,1,1,0.000916,0.998926,0.000157
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS187,1,1,0.216843,0.568123,0.215034
604,p0040presabsSTCC_qual,SR1287,0,1,0.418802,0.553160,0.028038
605,p0040presabsSTCC_qual,CFBRSa50,0,0,0.948100,0.035031,0.016869
606,p0040presabsSTCC_qual,NRS196,2,2,0.000964,0.039095,0.959940


In [19]:
y_prob = df_proba[df_proba['phage']=='p0040kpresabs_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[0.04047949, 0.28049988, 0.67902064],
       [0.17614147, 0.7572981 , 0.06656041],
       [0.15553288, 0.366461  , 0.47800606],
       [0.12364565, 0.29397324, 0.5823812 ],
       [0.48939386, 0.2195914 , 0.29101476],
       [0.4320851 , 0.3419223 , 0.22599259],
       [0.335571  , 0.3187999 , 0.34562907],
       [0.64074725, 0.15836829, 0.20088443],
       [0.04583629, 0.33266762, 0.621496  ],
       [0.44947666, 0.3355829 , 0.21494046],
       [0.49329385, 0.18378085, 0.32292533],
       [0.02040344, 0.36229795, 0.6172986 ],
       [0.65604544, 0.23389927, 0.11005541],
       [0.4982258 , 0.18142939, 0.32034487],
       [0.18078317, 0.535986  , 0.2832308 ],
       [0.23156622, 0.26814198, 0.50029176],
       [0.3936193 , 0.3453653 , 0.26101547],
       [0.15511893, 0.3227845 , 0.5220966 ],
       [0.04047949, 0.28049988, 0.67902064],
       [0.3936193 , 0.3453653 , 0.26101547],
       [0.49329385, 0.18378085, 0.32292533],
       [0.22725324, 0.3236626 , 0.44908413],
       [0.

In [20]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [21]:
ovo1 = rocauc_ovo(y_test, y_prob, average="macro", multi_class="ovo")
ovo1

0.743354525919531

In [22]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [23]:
ovr1 = rocauc_ovr(y_test, y_prob, average="macro", multi_class="ovr")
ovr1

0.743354525919531

In [24]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y)

In [25]:
dat2 = pd.DataFrame(X_test.iloc[:,0])
dat2['test'] = y_test

In [26]:
dat2

,id,test
165,NRS191,2
237,SR1287,0
243,SR3569,2
128,NRS100,0
107,NRS001,1
...,...,...
220,NRS265,1
233,NY439,2
68,CFBRSa05,0
175,NRS205,2


In [27]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [28]:
model2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [33]:
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [34]:
model2.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 740us/step - loss: 1.1445 - accuracy: 0.3164 - val_loss: 1.0819 - val_accuracy: 0.3684
Epoch 2/100
177/177 [==============================] - 0s 92us/step - loss: 1.0806 - accuracy: 0.3842 - val_loss: 1.0327 - val_accuracy: 0.4605
Epoch 3/100
177/177 [==============================] - 0s 186us/step - loss: 1.0665 - accuracy: 0.4181 - val_loss: 1.0172 - val_accuracy: 0.5132
Epoch 4/100
177/177 [==============================] - 0s 154us/step - loss: 1.0540 - accuracy: 0.4294 - val_loss: 1.0129 - val_accuracy: 0.5132
Epoch 5/100
177/177 [==============================] - 0s 190us/step - loss: 1.0420 - accuracy: 0.4407 - val_loss: 1.0087 - val_accuracy: 0.4868
Epoch 6/100
177/177 [==============================] - 0s 234us/step - loss: 1.0335 - accuracy: 0.4294 - val_loss: 1.0047 - val_accuracy: 0.5000
Epoch 7/100
177/177 [==============================] - 0s 178us/step - loss: 1.0283 - 

Epoch 57/100
177/177 [==============================] - 0s 118us/step - loss: 0.8351 - accuracy: 0.6271 - val_loss: 0.9473 - val_accuracy: 0.5395
Epoch 58/100
177/177 [==============================] - 0s 82us/step - loss: 0.8283 - accuracy: 0.6271 - val_loss: 0.9511 - val_accuracy: 0.5263
Epoch 59/100
177/177 [==============================] - 0s 84us/step - loss: 0.8218 - accuracy: 0.6328 - val_loss: 0.9523 - val_accuracy: 0.5658
Epoch 60/100
177/177 [==============================] - 0s 91us/step - loss: 0.8258 - accuracy: 0.6271 - val_loss: 0.9372 - val_accuracy: 0.5921
Epoch 61/100
177/177 [==============================] - 0s 133us/step - loss: 0.8226 - accuracy: 0.6497 - val_loss: 0.9626 - val_accuracy: 0.5526
Epoch 62/100
177/177 [==============================] - 0s 130us/step - loss: 0.8172 - accuracy: 0.6384 - val_loss: 0.9566 - val_accuracy: 0.5395
Epoch 63/100
177/177 [==============================] - 0s 119us/step - loss: 0.8111 - accuracy: 0.6384 - val_loss: 0.9440 - va

In [86]:
acc_test2 = model2.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test2*100))

76/76 [==============================] - 0s 135us/step
test accuracy: 48.68%


In [35]:
pred2 = model2.predict_classes(X_test)
pred2

array([2, 0, 2, 1, 0, 1, 1, 2, 2, 1, 0, 0, 0, 2, 0, 2, 2, 2, 2, 2, 1, 0,
       0, 1, 0, 1, 1, 0, 2, 0, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1,
       1, 1, 0, 1, 2, 0, 0, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 1, 0,
       2, 0, 2, 1, 1, 2, 0, 1, 2, 2])

In [36]:
dat2['pred'] = pred2
dat2

,id,test,pred
165,NRS191,2,2
237,SR1287,0,0
243,SR3569,2,2
128,NRS100,0,1
107,NRS001,1,0
...,...,...,...
220,NRS265,1,2
233,NY439,2,0
68,CFBRSa05,0,1
175,NRS205,2,2


In [37]:
proba2 = model2.predict_proba(X_test)
dat_proba2 = pd.DataFrame(proba2)

In [38]:
dat_proba2

,0,1,2
0,0.027459,0.243888,0.728654
1,0.410315,0.386324,0.203362
2,0.173924,0.278553,0.547522
3,0.400569,0.464484,0.134947
4,0.567512,0.405903,0.026585
...,...,...,...
71,0.051800,0.361655,0.586545
72,0.387840,0.320942,0.291218
73,0.356108,0.396302,0.247590
74,0.025135,0.374063,0.600802


In [39]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [40]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p40.csv", index = False,
         header=None)

In [90]:
hist2 = model2.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 143us/step - loss: 0.7200 - accuracy: 0.7006 - val_loss: 1.0241 - val_accuracy: 0.4474
Epoch 2/100
177/177 [==============================] - 0s 156us/step - loss: 0.7177 - accuracy: 0.7062 - val_loss: 1.0113 - val_accuracy: 0.4737
Epoch 3/100
177/177 [==============================] - 0s 175us/step - loss: 0.7139 - accuracy: 0.7232 - val_loss: 1.0095 - val_accuracy: 0.4868
Epoch 4/100
177/177 [==============================] - 0s 151us/step - loss: 0.7149 - accuracy: 0.7175 - val_loss: 1.0138 - val_accuracy: 0.4737
Epoch 5/100
177/177 [==============================] - 0s 154us/step - loss: 0.7292 - accuracy: 0.7119 - val_loss: 1.0303 - val_accuracy: 0.4605
Epoch 6/100
177/177 [==============================] - 0s 146us/step - loss: 0.7155 - accuracy: 0.7119 - val_loss: 1.0111 - val_accuracy: 0.4605
Epoch 7/100
177/177 [==============================] - 0s 165us/step - loss: 0.7117 -

Epoch 57/100
177/177 [==============================] - 0s 946us/step - loss: 0.6324 - accuracy: 0.7514 - val_loss: 1.0678 - val_accuracy: 0.4474
Epoch 58/100
177/177 [==============================] - 0s 136us/step - loss: 0.6338 - accuracy: 0.7684 - val_loss: 1.0746 - val_accuracy: 0.4342
Epoch 59/100
177/177 [==============================] - 0s 358us/step - loss: 0.6271 - accuracy: 0.7514 - val_loss: 1.0805 - val_accuracy: 0.4605
Epoch 60/100
177/177 [==============================] - 0s 133us/step - loss: 0.6312 - accuracy: 0.7627 - val_loss: 1.0937 - val_accuracy: 0.4342
Epoch 61/100
177/177 [==============================] - 0s 167us/step - loss: 0.6269 - accuracy: 0.7684 - val_loss: 1.0854 - val_accuracy: 0.4342
Epoch 62/100
177/177 [==============================] - 0s 270us/step - loss: 0.6257 - accuracy: 0.7571 - val_loss: 1.0927 - val_accuracy: 0.4342
Epoch 63/100
177/177 [==============================] - 0s 262us/step - loss: 0.6301 - accuracy: 0.7684 - val_loss: 1.1107 -

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.148005). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


In [91]:
print('train accuracy: %.2f%%' % (np.mean(hist2.history['accuracy'])*100))

train accuracy: 74.70%


In [29]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [30]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS210,2,0,0.999887,0.000112,8.851192e-07
1,p0017kpresabs_qual,Grady1,0,0,0.625329,0.369782,4.889404e-03
2,p0017kpresabs_qual,CFBRSa29,2,0,0.999098,0.000269,6.335156e-04
3,p0017kpresabs_qual,CFBRSa03,0,0,0.647338,0.331796,2.086646e-02
4,p0017kpresabs_qual,217,1,0,0.613342,0.381903,4.754707e-03
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS265,1,1,0.025601,0.687962,2.864372e-01
604,p0040presabsSTCC_qual,NY439,2,2,0.161947,0.266501,5.715521e-01
605,p0040presabsSTCC_qual,CFBRSa05,0,0,0.652983,0.254172,9.284494e-02
606,p0040presabsSTCC_qual,NRS205,2,2,0.000927,0.033760,9.653131e-01


In [31]:
y_prob2 = df_proba2[df_proba2['phage']=='p0040kpresabs_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[0.02745896, 0.24388753, 0.72865355],
       [0.41031465, 0.3863236 , 0.20336173],
       [0.17392445, 0.27855307, 0.5475225 ],
       [0.4005692 , 0.46448374, 0.13494709],
       [0.5675118 , 0.4059035 , 0.02658472],
       [0.26331708, 0.39223802, 0.34444493],
       [0.35610828, 0.39630225, 0.24758951],
       [0.0430968 , 0.06405223, 0.89285094],
       [0.13600153, 0.41564044, 0.448358  ],
       [0.23632129, 0.39817488, 0.36550388],
       [0.4429673 , 0.33354175, 0.22349092],
       [0.48203686, 0.35466257, 0.1633005 ],
       [0.48203686, 0.35466257, 0.1633005 ],
       [0.06772693, 0.18584412, 0.74642897],
       [0.49116248, 0.2417376 , 0.26709998],
       [0.16175167, 0.29743174, 0.5408166 ],
       [0.16274643, 0.12282115, 0.7144324 ],
       [0.16113858, 0.25844392, 0.5804175 ],
       [0.16175167, 0.29743174, 0.5408166 ],
       [0.17020357, 0.39619341, 0.43360302],
       [0.35610828, 0.39630225, 0.24758951],
       [0.42929918, 0.30019745, 0.27050337],
       [0.

In [32]:
ovo2 = rocauc_ovo(y_test, y_prob2, average="macro", multi_class="ovo")
ovo2

0.7134260852403562

In [33]:
ovr2 = rocauc_ovr(y_test, y_prob2, average="macro", multi_class="ovr")
ovr2

0.7134260852403562

In [34]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y)

In [35]:
dat3 = pd.DataFrame(X_test.iloc[:,0])
dat3['test'] = y_test

In [36]:
dat3

,id,test
165,NRS191,2
64,CFBREBSa135,0
121,NRS064,1
228,NY224,1
114,NRS035,1
...,...,...
16,BCH-SA-01,0
13,504,0
96,GA27,2
177,NRS209,1


In [37]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [38]:
model3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [46]:
model3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [47]:
model3.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 775us/step - loss: 1.1174 - accuracy: 0.3955 - val_loss: 1.0601 - val_accuracy: 0.3816
Epoch 2/100
177/177 [==============================] - 0s 134us/step - loss: 1.0623 - accuracy: 0.4237 - val_loss: 1.0731 - val_accuracy: 0.3816
Epoch 3/100
177/177 [==============================] - 0s 199us/step - loss: 1.0453 - accuracy: 0.4237 - val_loss: 1.0776 - val_accuracy: 0.3947
Epoch 4/100
177/177 [==============================] - 0s 158us/step - loss: 1.0330 - accuracy: 0.4463 - val_loss: 1.0691 - val_accuracy: 0.4079
Epoch 5/100
177/177 [==============================] - 0s 195us/step - loss: 1.0160 - accuracy: 0.4576 - val_loss: 1.0654 - val_accuracy: 0.3816
Epoch 6/100
177/177 [==============================] - 0s 166us/step - loss: 1.0067 - accuracy: 0.4520 - val_loss: 1.0591 - val_accuracy: 0.3684
Epoch 7/100
177/177 [==============================] - 0s 145us/step - loss: 1.0026 -

Epoch 57/100
177/177 [==============================] - 0s 99us/step - loss: 0.8124 - accuracy: 0.6497 - val_loss: 1.0630 - val_accuracy: 0.5000
Epoch 58/100
177/177 [==============================] - 0s 95us/step - loss: 0.8130 - accuracy: 0.6554 - val_loss: 1.0755 - val_accuracy: 0.5132
Epoch 59/100
177/177 [==============================] - 0s 88us/step - loss: 0.8055 - accuracy: 0.6723 - val_loss: 1.0764 - val_accuracy: 0.5132
Epoch 60/100
177/177 [==============================] - 0s 122us/step - loss: 0.8028 - accuracy: 0.6667 - val_loss: 1.0712 - val_accuracy: 0.5263
Epoch 61/100
177/177 [==============================] - 0s 93us/step - loss: 0.8007 - accuracy: 0.6667 - val_loss: 1.0833 - val_accuracy: 0.5132
Epoch 62/100
177/177 [==============================] - 0s 109us/step - loss: 0.8003 - accuracy: 0.6667 - val_loss: 1.0846 - val_accuracy: 0.5132
Epoch 63/100
177/177 [==============================] - 0s 95us/step - loss: 0.7955 - accuracy: 0.6780 - val_loss: 1.0843 - val_

In [99]:
acc_test3 = model3.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test3*100))

76/76 [==============================] - 0s 128us/step
test accuracy: 51.32%


In [48]:
pred3 = model3.predict_classes(X_test)
pred3

array([2, 2, 1, 2, 2, 0, 0, 0, 2, 1, 2, 0, 1, 2, 1, 1, 2, 1, 2, 1, 2, 0,
       0, 0, 2, 2, 1, 1, 0, 1, 1, 2, 1, 0, 2, 1, 0, 0, 0, 2, 1, 2, 2, 2,
       2, 1, 0, 0, 0, 1, 2, 1, 2, 0, 2, 0, 1, 2, 1, 0, 1, 0, 1, 1, 0, 2,
       1, 0, 2, 2, 1, 0, 0, 1, 1, 0])

In [49]:
dat3['pred'] = pred3
dat3

,id,test,pred
165,NRS191,2,2
64,CFBREBSa135,0,2
121,NRS064,1,1
228,NY224,1,2
114,NRS035,1,2
...,...,...,...
16,BCH-SA-01,0,0
13,504,0,0
96,GA27,2,1
177,NRS209,1,1


In [50]:
proba3 = model3.predict_proba(X_test)
dat_proba3 = pd.DataFrame(proba3)

In [51]:
dat_proba3

,0,1,2
0,0.059193,0.221468,0.719339
1,0.347444,0.230777,0.421778
2,0.030019,0.960383,0.009598
3,0.142872,0.296726,0.560402
4,0.124767,0.405024,0.470210
...,...,...,...
71,0.512167,0.306114,0.181720
72,0.469700,0.363842,0.166458
73,0.290176,0.535882,0.173942
74,0.255762,0.649710,0.094527


In [52]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [53]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p40.csv", index = False,
         header=None)

In [103]:
hist3 = model3.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 554us/step - loss: 0.7323 - accuracy: 0.7006 - val_loss: 1.0780 - val_accuracy: 0.5132
Epoch 2/100
177/177 [==============================] - 0s 200us/step - loss: 0.7283 - accuracy: 0.6949 - val_loss: 1.0694 - val_accuracy: 0.5263
Epoch 3/100
177/177 [==============================] - 0s 151us/step - loss: 0.7281 - accuracy: 0.6949 - val_loss: 1.0592 - val_accuracy: 0.5395
Epoch 4/100
177/177 [==============================] - 0s 152us/step - loss: 0.7273 - accuracy: 0.6893 - val_loss: 1.0740 - val_accuracy: 0.5000
Epoch 5/100
177/177 [==============================] - 0s 146us/step - loss: 0.7256 - accuracy: 0.7006 - val_loss: 1.0875 - val_accuracy: 0.5263
Epoch 6/100
177/177 [==============================] - 0s 124us/step - loss: 0.7263 - accuracy: 0.7006 - val_loss: 1.0830 - val_accuracy: 0.5395
Epoch 7/100
177/177 [==============================] - 0s 122us/step - loss: 0.7253 -

Epoch 57/100
177/177 [==============================] - 0s 255us/step - loss: 0.6428 - accuracy: 0.7401 - val_loss: 1.1577 - val_accuracy: 0.4868
Epoch 58/100
177/177 [==============================] - 0s 206us/step - loss: 0.6477 - accuracy: 0.7514 - val_loss: 1.1600 - val_accuracy: 0.5000
Epoch 59/100
177/177 [==============================] - 0s 180us/step - loss: 0.6405 - accuracy: 0.7514 - val_loss: 1.1572 - val_accuracy: 0.4868
Epoch 60/100
177/177 [==============================] - 0s 405us/step - loss: 0.6382 - accuracy: 0.7514 - val_loss: 1.1642 - val_accuracy: 0.4474
Epoch 61/100
177/177 [==============================] - 0s 165us/step - loss: 0.6383 - accuracy: 0.7514 - val_loss: 1.1665 - val_accuracy: 0.5000
Epoch 62/100
177/177 [==============================] - 0s 163us/step - loss: 0.6455 - accuracy: 0.7401 - val_loss: 1.1611 - val_accuracy: 0.5263
Epoch 63/100
177/177 [==============================] - 0s 141us/step - loss: 0.6343 - accuracy: 0.7345 - val_loss: 1.1685 -

In [104]:
print('train accuracy: %.2f%%' % (np.mean(hist3.history['accuracy'])*100))

train accuracy: 73.68%


In [39]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [40]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NY360,2,2,2.165526e-02,4.848140e-01,0.493531
1,p0017kpresabs_qual,EUH25,2,0,9.986388e-01,1.245148e-03,0.000116
2,p0017kpresabs_qual,EUH15,2,2,9.227520e-04,1.424882e-02,0.984828
3,p0017kpresabs_qual,NRS241,0,0,8.374333e-01,1.614128e-01,0.001154
4,p0017kpresabs_qual,SR2852,2,2,3.976981e-09,5.145955e-10,1.000000
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,BCH-SA-01,0,0,9.305407e-01,6.356251e-02,0.005897
604,p0040presabsSTCC_qual,504,0,1,4.476389e-02,8.577548e-01,0.097481
605,p0040presabsSTCC_qual,GA27,2,1,2.779456e-01,5.384378e-01,0.183617
606,p0040presabsSTCC_qual,NRS209,1,0,4.210180e-01,3.559393e-01,0.223043


In [41]:
y_prob3 = df_proba3[df_proba3['phage']=='p0040kpresabs_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[0.05919308, 0.22146776, 0.71933913],
       [0.3474444 , 0.23077743, 0.42177823],
       [0.03001855, 0.9603833 , 0.00959818],
       [0.14287177, 0.29672587, 0.5604024 ],
       [0.12476666, 0.4050238 , 0.47020954],
       [0.4492654 , 0.17811875, 0.3726158 ],
       [0.38617927, 0.30036715, 0.3134536 ],
       [0.6480597 , 0.3371173 , 0.01482301],
       [0.08153449, 0.37896878, 0.53949666],
       [0.25451064, 0.40364456, 0.34184483],
       [0.18137051, 0.16252472, 0.6561048 ],
       [0.38617927, 0.30036715, 0.3134536 ],
       [0.10315937, 0.5849639 , 0.31187674],
       [0.03209526, 0.26224244, 0.7056623 ],
       [0.06595705, 0.5413944 , 0.39264855],
       [0.0579153 , 0.9104134 , 0.0316714 ],
       [0.10755979, 0.31743598, 0.5750042 ],
       [0.02664254, 0.9655366 , 0.00782085],
       [0.10701442, 0.39919853, 0.49378714],
       [0.05235415, 0.9157013 , 0.03194448],
       [0.25403908, 0.26799777, 0.47796315],
       [0.48178965, 0.19618548, 0.32202485],
       [0.

In [44]:
ovo3 = rocauc_ovo(y_test, y_prob3, average="macro", multi_class="ovo")
ovo3

0.6248976736619902

In [45]:
ovr3 = rocauc_ovr(y_test, y_prob3, average="macro", multi_class="ovr")
ovr3

0.6248976736619902

In [46]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y)

In [47]:
dat4 = pd.DataFrame(X_test.iloc[:,0])
dat4['test'] = y_test

In [48]:
dat4

,id,test
54,CFBREBSa121,1
28,BCH-SA-13,1
143,NRS148,2
73,CFBRSa23,2
57,CFBREBSa125,0
...,...,...
208,NRS247,1
183,NRS215,1
248,SR4152,2
114,NRS035,1


In [49]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [50]:
model4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [59]:
model4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [60]:
model4.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 793us/step - loss: 1.0764 - accuracy: 0.3559 - val_loss: 1.0954 - val_accuracy: 0.3816
Epoch 2/100
177/177 [==============================] - 0s 122us/step - loss: 1.0378 - accuracy: 0.3898 - val_loss: 1.0837 - val_accuracy: 0.4342
Epoch 3/100
177/177 [==============================] - 0s 142us/step - loss: 1.0249 - accuracy: 0.4407 - val_loss: 1.0735 - val_accuracy: 0.4605
Epoch 4/100
177/177 [==============================] - 0s 232us/step - loss: 1.0085 - accuracy: 0.4915 - val_loss: 1.0738 - val_accuracy: 0.5000
Epoch 5/100
177/177 [==============================] - 0s 172us/step - loss: 0.9973 - accuracy: 0.5085 - val_loss: 1.0720 - val_accuracy: 0.4868
Epoch 6/100
177/177 [==============================] - 0s 197us/step - loss: 0.9892 - accuracy: 0.5537 - val_loss: 1.0731 - val_accuracy: 0.4474
Epoch 7/100
177/177 [==============================] - 0s 154us/step - loss: 0.9799 -

Epoch 57/100
177/177 [==============================] - 0s 175us/step - loss: 0.7967 - accuracy: 0.6441 - val_loss: 1.1123 - val_accuracy: 0.4342
Epoch 58/100
177/177 [==============================] - 0s 157us/step - loss: 0.7955 - accuracy: 0.6384 - val_loss: 1.1109 - val_accuracy: 0.4079
Epoch 59/100
177/177 [==============================] - 0s 171us/step - loss: 0.7926 - accuracy: 0.6610 - val_loss: 1.1266 - val_accuracy: 0.4342
Epoch 60/100
177/177 [==============================] - 0s 100us/step - loss: 0.7905 - accuracy: 0.6497 - val_loss: 1.1302 - val_accuracy: 0.4211
Epoch 61/100
177/177 [==============================] - 0s 214us/step - loss: 0.7934 - accuracy: 0.6554 - val_loss: 1.1280 - val_accuracy: 0.4342
Epoch 62/100
177/177 [==============================] - 0s 160us/step - loss: 0.7866 - accuracy: 0.6554 - val_loss: 1.1371 - val_accuracy: 0.4211
Epoch 63/100
177/177 [==============================] - 0s 177us/step - loss: 0.7850 - accuracy: 0.6441 - val_loss: 1.1367 -

In [129]:
acc_test4 = model4.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test4*100))

76/76 [==============================] - 0s 189us/step
test accuracy: 40.79%


In [61]:
pred4 = model4.predict_classes(X_test)
pred4

array([2, 0, 2, 2, 0, 0, 2, 0, 0, 2, 1, 1, 0, 0, 0, 1, 0, 1, 1, 2, 0, 2,
       0, 0, 2, 0, 1, 0, 2, 2, 1, 0, 2, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2,
       0, 0, 0, 1, 2, 0, 0, 2, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 2, 1,
       1, 2, 0, 2, 1, 1, 1, 2, 2, 2])

In [62]:
dat4['pred'] = pred4
dat4

,id,test,pred
54,CFBREBSa121,1,2
28,BCH-SA-13,1,0
143,NRS148,2,2
73,CFBRSa23,2,2
57,CFBREBSa125,0,0
...,...,...,...
208,NRS247,1,1
183,NRS215,1,1
248,SR4152,2,2
114,NRS035,1,2


In [63]:
proba4 = model4.predict_proba(X_test)
dat_proba4 = pd.DataFrame(proba4)

In [64]:
dat_proba4

,0,1,2
0,0.165234,0.331337,0.503428
1,0.502827,0.256943,0.240230
2,0.047732,0.225732,0.726536
3,0.178010,0.292197,0.529794
4,0.502827,0.256943,0.240230
...,...,...,...
71,0.198291,0.539802,0.261907
72,0.009701,0.518846,0.471453
73,0.057430,0.079704,0.862866
74,0.163357,0.341239,0.495404


In [65]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [66]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p40.csv", index = False,
         header=None)

In [133]:
hist4 = model4.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 220us/step - loss: 0.7007 - accuracy: 0.7175 - val_loss: 1.2247 - val_accuracy: 0.4079
Epoch 2/100
177/177 [==============================] - 0s 253us/step - loss: 0.7053 - accuracy: 0.7119 - val_loss: 1.2515 - val_accuracy: 0.4342
Epoch 3/100
177/177 [==============================] - 0s 245us/step - loss: 0.7023 - accuracy: 0.7062 - val_loss: 1.2214 - val_accuracy: 0.3947
Epoch 4/100
177/177 [==============================] - 0s 203us/step - loss: 0.6931 - accuracy: 0.7062 - val_loss: 1.2390 - val_accuracy: 0.4342
Epoch 5/100
177/177 [==============================] - 0s 171us/step - loss: 0.6882 - accuracy: 0.7119 - val_loss: 1.2419 - val_accuracy: 0.3947
Epoch 6/100
177/177 [==============================] - 0s 153us/step - loss: 0.6902 - accuracy: 0.7288 - val_loss: 1.2470 - val_accuracy: 0.4079
Epoch 7/100
177/177 [==============================] - 0s 155us/step - loss: 0.6841 -

Epoch 57/100
177/177 [==============================] - 0s 167us/step - loss: 0.6187 - accuracy: 0.7684 - val_loss: 1.3415 - val_accuracy: 0.4342
Epoch 58/100
177/177 [==============================] - 0s 132us/step - loss: 0.6191 - accuracy: 0.7571 - val_loss: 1.3448 - val_accuracy: 0.4342
Epoch 59/100
177/177 [==============================] - 0s 144us/step - loss: 0.6143 - accuracy: 0.7684 - val_loss: 1.3338 - val_accuracy: 0.4474
Epoch 60/100
177/177 [==============================] - 0s 149us/step - loss: 0.6192 - accuracy: 0.7571 - val_loss: 1.3419 - val_accuracy: 0.4342
Epoch 61/100
177/177 [==============================] - 0s 172us/step - loss: 0.6146 - accuracy: 0.7571 - val_loss: 1.3655 - val_accuracy: 0.4342
Epoch 62/100
177/177 [==============================] - 0s 189us/step - loss: 0.6212 - accuracy: 0.7514 - val_loss: 1.3467 - val_accuracy: 0.4342
Epoch 63/100
177/177 [==============================] - 0s 210us/step - loss: 0.6148 - accuracy: 0.7458 - val_loss: 1.3617 -

In [134]:
print('train accuracy: %.2f%%' % (np.mean(hist4.history['accuracy'])*100))

train accuracy: 74.54%


In [51]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [52]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,SR1129,2,2,4.821690e-07,0.000001,9.999983e-01
1,p0017kpresabs_qual,CA105,2,2,2.652370e-03,0.000049,9.972990e-01
2,p0017kpresabs_qual,NRS175,1,1,1.198157e-02,0.988018,5.232074e-09
3,p0017kpresabs_qual,EUH25,2,1,1.388957e-01,0.735954,1.251503e-01
4,p0017kpresabs_qual,NRS070,2,1,7.231966e-03,0.810317,1.824512e-01
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS247,1,2,2.957360e-02,0.385730,5.846961e-01
604,p0040presabsSTCC_qual,NRS215,1,2,3.196635e-03,0.445057,5.517461e-01
605,p0040presabsSTCC_qual,SR4152,2,2,4.319404e-02,0.028984,9.278219e-01
606,p0040presabsSTCC_qual,NRS035,1,1,4.902312e-03,0.680427,3.146706e-01


In [53]:
y_prob4 = df_proba4[df_proba4['phage']=='p0040kpresabs_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[0.16523431, 0.33133727, 0.5034284 ],
       [0.502827  , 0.25694343, 0.24022952],
       [0.04773205, 0.22573198, 0.7265359 ],
       [0.17800961, 0.29219654, 0.5297938 ],
       [0.502827  , 0.25694343, 0.24022952],
       [0.5390633 , 0.29135808, 0.16957867],
       [0.05742967, 0.07970402, 0.8628663 ],
       [0.8313017 , 0.05558755, 0.11311081],
       [0.5390633 , 0.29135808, 0.16957867],
       [0.03014656, 0.4413388 , 0.5285147 ],
       [0.02700538, 0.71684307, 0.2561516 ],
       [0.02916371, 0.64990294, 0.32093334],
       [0.58799314, 0.22570574, 0.18630114],
       [0.45778385, 0.2738882 , 0.26832792],
       [0.5359039 , 0.14224726, 0.3218489 ],
       [0.36572504, 0.4988753 , 0.13539974],
       [0.45986798, 0.37405753, 0.16607451],
       [0.19829066, 0.5398022 , 0.26190707],
       [0.1412882 , 0.67800796, 0.1807038 ],
       [0.17800961, 0.29219654, 0.5297938 ],
       [0.5390633 , 0.29135808, 0.16957867],
       [0.18196003, 0.3765768 , 0.4414631 ],
       [0.

In [54]:
ovo4 = rocauc_ovo(y_test, y_prob4, average="macro", multi_class="ovo")
ovo4

0.5955877204333735

In [55]:
ovr4 = rocauc_ovr(y_test, y_prob4, average="macro", multi_class="ovr")
ovr4

0.5955877204333735

In [56]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.6693165013138127

In [57]:
np.std(ovos)

0.06090207275160645

In [58]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.6693165013138127

In [59]:
np.std(ovrs)

0.06090207275160645

In [135]:
accs = [acc_test1, acc_test2, acc_test3, acc_test4]

In [136]:
mean = np.mean(accs)
print('test accuracy mean: %.2f%%' % (mean*100))

test accuracy mean: 48.03%


In [137]:
std = np.std(accs)
print('test accuracy standard deviation:', std)

test accuracy standard deviation: 0.04314104878345486


In [138]:
accs_train = [np.mean(hist1.history['accuracy']), np.mean(hist2.history['accuracy']), np.mean(hist3.history['accuracy']),
             np.mean(hist4.history['accuracy'])]

In [139]:
mean_train = np.mean(accs_train)
print('train accuracy mean: %.2f%%' % (mean_train*100))

train accuracy mean: 74.02%


In [140]:
std_train = np.std(accs_train)
print('train accuracy standard deviation:', std_train)

train accuracy standard deviation: 0.006239113


In [108]:
############ Feature selection using lasso ##########

In [60]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [61]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
selection.fit(X.loc[:, X.columns != 'id'], y)

SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='auto',
                                             n_jobs=None, penalty='l1',
                                             random_state=None,
                                             solver='liblinear', tol=0.0001,
                                             verbose=0, warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [62]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [63]:
X_train_features = np.vstack((names, X.loc[:, X.columns != 'id']))
X_train_features = pd.DataFrame(X_train_features)

In [64]:
sel_features = X_train_features.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 180
selected features: 53


In [65]:
cols = sel_features.values
cols.reshape((1, -1))

array([[  0,   7,  12,  21,  35,  41,  44,  62,  64,  66,  67,  68,  70,
         71,  75,  79,  80,  81,  82,  83,  87,  88,  90,  91,  92,  93,
         94, 107, 108, 109, 110, 113, 116, 117, 125, 126, 129, 131, 132,
        134, 135, 136, 139, 143, 146, 148, 152, 153, 163, 169, 174, 175,
        179]])

In [66]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT',
       'TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC',
       'TTTCACTGCCTGT', 'TTGCAAATCCAC',
       'TTAAAACAGCACAAACTGCTCATACAGTTAAAACAGCACAAACTGCTCAAGAACAAAATAAAGTTCAAACACCTGTTAAAGATGTTGCAACAGCGAAATC',
       'TGTCTGATTTTTT',
       'TGGCTTAGGCTATAAATTGGAAAGACCTAAAGAACAATGATGAAGTTTCACCACCGCTTAATGTTATTGATAAGCAGTATATTAATTATCAGTTTTGTTA',
       'TCGTGTCTGT', 'TCGCTGAAATAT', 'TCGCTGAAATATTT', 'TCGCTGAAATATTTG',
       'TCGCTGAAATATTTGCGACAT', 'TCATCAAACTTT', 'TCAGTAGAGAT',
       'TCACACCGCCT', 'TATCAGTTTTGTT', 'TATAGTGTTCAT', 'TAGTCATACAAT',
       'TAGCTAAATCC', 'TAGATTCAAATAT', 'TAATGGTAGTAGATAATTTTTC',
       'TAATCTTGTTGTT',
       'TAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTGATT',
       'TAACAGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTT

In [67]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']

In [68]:
df_sel['strain'] = X.iloc[:,0]

In [69]:
df_sel

,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,TTTCACTGCCTGT,TTGCAAATCCAC,TTAAAACAGCACAAACTGCTCATACAGTTAAAACAGCACAAACTGCTCAAGAACAAAATAAAGTTCAAACACCTGTTAAAGATGTTGCAACAGCGAAATC,TGTCTGATTTTTT,TGGCTTAGGCTATAAATTGGAAAGACCTAAAGAACAATGATGAAGTTTCACCACCGCTTAATGTTATTGATAAGCAGTATATTAATTATCAGTTTTGTTA,TCGTGTCTGT,TCGCTGAAATAT,TCGCTGAAATATTT,...,CTAATCCTTCAAT,CGCTGAAATATTTG,CGCTGAAATATTTGCG,CAAAAGATCAAACTAAAACACAAACTGCTCATACAGTTAAAACAGCACAAACTGCTCATACAGTTAAAACAGCACAAACTGCTCAAGAACAAAATAAAGT,AGTTTATCTGCT,ACTTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTT,ACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTGATTGT,AACTGCTCATACAGTTAAAACAGCACAAACTGCTCATACAGTTAAAACAGCACAAACTGCTCAAGAACAAAATAAAGTTCAAACACCTGTTAAAGATGTT,pheno,strain
0,1,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,1,0,0,2,107
1,1,0,1,1,0,1,1,1,1,1,...,1,1,1,0,1,1,0,0,0,109
2,0,0,1,1,0,1,1,1,1,1,...,1,1,1,0,1,0,0,0,2,115
3,0,1,1,1,1,1,0,1,1,1,...,1,1,1,1,1,0,1,1,2,120335
4,0,1,1,1,1,1,0,1,1,1,...,1,1,1,1,1,0,1,1,2,120337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0,1,1,1,1,1,0,1,1,1,...,1,1,1,1,1,0,1,1,2,SR4152
249,0,1,1,1,1,1,0,1,1,1,...,1,1,1,1,1,0,1,1,1,SR4153
250,0,0,1,1,0,1,1,1,1,1,...,1,1,1,0,1,0,0,0,1,SR4155
251,1,1,0,1,1,0,0,1,1,1,...,0,1,1,1,0,1,1,1,2,SR4156


In [70]:
X_sel = df_sel.loc[:, df_sel.columns != 'pheno']
y_sel = df_sel['pheno']
print(X_sel.shape, y_sel.shape, df_sel.shape)

(253, 54) (253,) (253, 55)


In [71]:
df_sel['pheno'].value_counts()

2    95
1    94
0    64
Name: pheno, dtype: int64

In [72]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=567,
                                                    stratify=y_sel)

In [73]:
dat5 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat5['test'] = y_sel_test

In [74]:
dat5

,strain,test
251,SR4156,2
17,BCH-SA-02,0
158,NRS180,2
232,NY417,1
47,CFBREBSa110,2
...,...,...
62,CFBREBSa131,2
138,NRS112,2
21,BCH-SA-06,0
15,834N,2


In [75]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [76]:
#### neural network on over-sampling data
model_sel = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [84]:
model_sel.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [85]:
model_sel.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 709us/step - loss: 1.1923 - accuracy: 0.2147 - val_loss: 1.1208 - val_accuracy: 0.3684
Epoch 2/100
177/177 [==============================] - 0s 85us/step - loss: 1.1612 - accuracy: 0.2825 - val_loss: 1.1036 - val_accuracy: 0.3553
Epoch 3/100
177/177 [==============================] - 0s 93us/step - loss: 1.1359 - accuracy: 0.3333 - val_loss: 1.0917 - val_accuracy: 0.3816
Epoch 4/100
177/177 [==============================] - 0s 148us/step - loss: 1.1124 - accuracy: 0.3503 - val_loss: 1.0835 - val_accuracy: 0.3947
Epoch 5/100
177/177 [==============================] - 0s 99us/step - loss: 1.0922 - accuracy: 0.3898 - val_loss: 1.0775 - val_accuracy: 0.3816
Epoch 6/100
177/177 [==============================] - 0s 92us/step - loss: 1.0730 - accuracy: 0.4237 - val_loss: 1.0714 - val_accuracy: 0.3816
Epoch 7/100
177/177 [==============================] - 0s 109us/step - loss: 1.0567 - acc

In [180]:
acc_test_sel = model_sel.evaluate(X_sel_test, y_sel_test)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_sel*100))

76/76 [==============================] - 0s 94us/step
over-sampling test accuracy: 42.11%


In [86]:
pred5 = model_sel.predict_classes(X_sel_test)
pred5

array([2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 0, 0, 2, 2, 2, 1,
       1, 0, 1, 1, 1, 0, 2, 1, 0, 2, 0, 1, 2, 0, 2, 2, 2, 1, 1, 1, 1, 1,
       2, 2, 2, 2, 2, 1, 1, 2, 0, 2, 2, 2, 1, 2, 1, 0, 1, 2, 2, 1, 2, 0,
       2, 2, 2, 2, 0, 0, 1, 1, 2, 2])

In [87]:
dat5['pred'] = pred5
dat5

,strain,test,pred
251,SR4156,2,2
17,BCH-SA-02,0,1
158,NRS180,2,2
232,NY417,1,2
47,CFBREBSa110,2,2
...,...,...,...
62,CFBREBSa131,2,0
138,NRS112,2,1
21,BCH-SA-06,0,1
15,834N,2,2


In [88]:
proba5 = model_sel.predict_proba(X_sel_test)
dat_proba5 = pd.DataFrame(proba5)

In [89]:
dat_proba5

,0,1,2
0,0.273738,0.233240,0.493022
1,0.291361,0.594861,0.113779
2,0.048533,0.328458,0.623009
3,0.150530,0.207708,0.641762
4,0.150530,0.207708,0.641762
...,...,...,...
71,0.479370,0.121359,0.399271
72,0.375478,0.541996,0.082526
73,0.291361,0.594861,0.113779
74,0.150530,0.207708,0.641762


In [90]:
dat_proba5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba5.csv", index = False,
         header=None)

In [91]:
dat5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/5p40.csv", index = False,
         header=None)

In [184]:
hist_sel = model_sel.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 120us/step - loss: 0.6450 - accuracy: 0.7514 - val_loss: 1.1442 - val_accuracy: 0.4211
Epoch 2/100
177/177 [==============================] - 0s 90us/step - loss: 0.6413 - accuracy: 0.7401 - val_loss: 1.1510 - val_accuracy: 0.4211
Epoch 3/100
177/177 [==============================] - 0s 120us/step - loss: 0.6402 - accuracy: 0.7232 - val_loss: 1.1596 - val_accuracy: 0.4079
Epoch 4/100
177/177 [==============================] - 0s 128us/step - loss: 0.6385 - accuracy: 0.7458 - val_loss: 1.1550 - val_accuracy: 0.3947
Epoch 5/100
177/177 [==============================] - 0s 115us/step - loss: 0.6370 - accuracy: 0.7514 - val_loss: 1.1469 - val_accuracy: 0.4342
Epoch 6/100
177/177 [==============================] - 0s 107us/step - loss: 0.6340 - accuracy: 0.7514 - val_loss: 1.1478 - val_accuracy: 0.4342
Epoch 7/100
177/177 [==============================] - 0s 102us/step - loss: 0.6347 - 

In [185]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel.history['accuracy'])*100))

train accuracy: 77.75%


In [77]:
df_proba5 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [78]:
df_proba5

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NY360,2,1,3.848032e-03,0.996096,0.000056
1,p0017kpresabs_qual,NRS113,0,0,8.441349e-01,0.000015,0.155850
2,p0017kpresabs_qual,120337,2,2,1.466081e-02,0.005713,0.979626
3,p0017kpresabs_qual,CFBREBSa127,1,0,9.985392e-01,0.000350,0.001111
4,p0017kpresabs_qual,GA27,2,2,1.665667e-08,0.000005,0.999995
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,CFBREBSa131,2,1,6.617341e-03,0.565843,0.427540
604,p0040presabsSTCC_qual,NRS112,2,2,1.055758e-03,0.045611,0.953333
605,p0040presabsSTCC_qual,BCH-SA-06,0,0,8.747076e-01,0.071162,0.054130
606,p0040presabsSTCC_qual,834N,2,2,2.593112e-06,0.000013,0.999985


In [79]:
y_prob5 = df_proba5[df_proba5['phage']=='p0040kpresabs_qual'].iloc[:,-3:]
y_prob5 = y_prob5.to_numpy()
y_prob5

array([[0.27373773, 0.23323973, 0.49302247],
       [0.29136065, 0.5948606 , 0.11377878],
       [0.04853262, 0.3284582 , 0.6230092 ],
       [0.15052983, 0.20770812, 0.6417621 ],
       [0.15052983, 0.20770812, 0.6417621 ],
       [0.15052983, 0.20770812, 0.6417621 ],
       [0.0366947 , 0.21462493, 0.74868035],
       [0.1073032 , 0.12761863, 0.7650782 ],
       [0.00427501, 0.1338416 , 0.8618834 ],
       [0.39074245, 0.5191766 , 0.09008092],
       [0.0222603 , 0.20988078, 0.76785886],
       [0.15052983, 0.20770812, 0.6417621 ],
       [0.10275727, 0.7645176 , 0.13272516],
       [0.32594627, 0.50115514, 0.17289868],
       [0.01726191, 0.42798263, 0.5547555 ],
       [0.14649852, 0.23868795, 0.6148135 ],
       [0.6088695 , 0.13168919, 0.2594413 ],
       [0.8687113 , 0.06762633, 0.06366237],
       [0.08201418, 0.18083654, 0.7371493 ],
       [0.1073032 , 0.12761863, 0.7650782 ],
       [0.15052983, 0.20770812, 0.6417621 ],
       [0.05884498, 0.511712  , 0.42944297],
       [0.

In [80]:
ovo5 = rocauc_ovo(y_sel_test, y_prob5, average="macro", multi_class="ovo")
ovo5

0.5858067082628343

In [81]:
ovr5 = rocauc_ovr(y_sel_test, y_prob5, average="macro", multi_class="ovr")
ovr5

0.5858067082628343

In [82]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=678,
                                                    stratify=y_sel)

In [83]:
dat6 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat6['test'] = y_sel_test

In [84]:
dat6

,strain,test
152,NRS169,1
210,NRS249,2
218,NRS262,2
238,SR1746,1
113,NRS029,1
...,...,...
96,GA27,2
95,GA231,2
237,SR1287,0
14,506,2


In [85]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [86]:
model_sel2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [97]:
model_sel2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [98]:
model_sel2.fit(X_sel_train, y_sel_train,
          batch_size=16, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 1ms/step - loss: 1.2035 - accuracy: 0.3390 - val_loss: 1.0867 - val_accuracy: 0.3947
Epoch 2/100
177/177 [==============================] - 0s 236us/step - loss: 1.1020 - accuracy: 0.4124 - val_loss: 1.0968 - val_accuracy: 0.3816
Epoch 3/100
177/177 [==============================] - 0s 209us/step - loss: 1.0902 - accuracy: 0.4068 - val_loss: 1.1113 - val_accuracy: 0.3421
Epoch 4/100
177/177 [==============================] - 0s 252us/step - loss: 1.0815 - accuracy: 0.4011 - val_loss: 1.0901 - val_accuracy: 0.3289
Epoch 5/100
177/177 [==============================] - 0s 209us/step - loss: 1.0613 - accuracy: 0.4181 - val_loss: 1.0576 - val_accuracy: 0.4079
Epoch 6/100
177/177 [==============================] - 0s 218us/step - loss: 1.0554 - accuracy: 0.3503 - val_loss: 1.0485 - val_accuracy: 0.3947
Epoch 7/100
177/177 [==============================] - 0s 198us/step - loss: 1.0400 - a

Epoch 57/100
177/177 [==============================] - 0s 243us/step - loss: 0.8374 - accuracy: 0.5989 - val_loss: 1.0497 - val_accuracy: 0.4474
Epoch 58/100
177/177 [==============================] - 0s 340us/step - loss: 0.8277 - accuracy: 0.6271 - val_loss: 1.0341 - val_accuracy: 0.4868
Epoch 59/100
177/177 [==============================] - 0s 283us/step - loss: 0.8308 - accuracy: 0.6271 - val_loss: 1.0495 - val_accuracy: 0.4868
Epoch 60/100
177/177 [==============================] - 0s 333us/step - loss: 0.8253 - accuracy: 0.6328 - val_loss: 1.0519 - val_accuracy: 0.4868
Epoch 61/100
177/177 [==============================] - 0s 408us/step - loss: 0.8198 - accuracy: 0.6271 - val_loss: 1.0375 - val_accuracy: 0.5132
Epoch 62/100
177/177 [==============================] - 0s 326us/step - loss: 0.8217 - accuracy: 0.6102 - val_loss: 1.0331 - val_accuracy: 0.4868
Epoch 63/100
177/177 [==============================] - 0s 155us/step - loss: 0.8172 - accuracy: 0.6271 - val_loss: 1.0434 -

In [201]:
acc_test_sel2 = model_sel2.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test_sel2*100))

76/76 [==============================] - 0s 151us/step
test accuracy: 50.00%


In [99]:
pred6 = model_sel2.predict_classes(X_sel_test)
pred6

array([1, 2, 1, 1, 1, 0, 1, 1, 1, 2, 0, 1, 0, 1, 2, 1, 2, 1, 1, 2, 2, 2,
       2, 2, 2, 2, 2, 0, 2, 0, 2, 1, 1, 1, 2, 2, 0, 1, 0, 0, 1, 2, 1, 2,
       0, 1, 0, 1, 2, 1, 1, 2, 2, 0, 2, 1, 0, 2, 2, 1, 0, 1, 0, 2, 0, 0,
       1, 2, 0, 2, 0, 1, 2, 0, 0, 0])

In [100]:
dat6['pred'] = pred6
dat6

,strain,test,pred
152,NRS169,1,1
210,NRS249,2,2
218,NRS262,2,1
238,SR1746,1,1
113,NRS029,1,1
...,...,...,...
96,GA27,2,1
95,GA231,2,2
237,SR1287,0,0
14,506,2,0


In [101]:
proba6 = model_sel2.predict_proba(X_sel_test)
dat_proba6 = pd.DataFrame(proba6)

In [102]:
dat_proba6

,0,1,2
0,0.051419,0.583905,0.364676
1,0.101089,0.312194,0.586718
2,0.315176,0.446782,0.238041
3,0.323659,0.433297,0.243044
4,0.352156,0.590625,0.057219
...,...,...,...
71,0.248237,0.442374,0.309390
72,0.267271,0.173350,0.559380
73,0.546218,0.215303,0.238479
74,0.453734,0.252130,0.294135


In [103]:
dat_proba6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba6.csv", index = False,
         header=None)

In [104]:
dat6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/6p40.csv", index = False,
         header=None)

In [205]:
hist_sel2 = model_sel2.fit(X_sel_train, y_sel_train,
          batch_size=16, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 206us/step - loss: 0.7525 - accuracy: 0.7006 - val_loss: 1.0553 - val_accuracy: 0.4605
Epoch 2/100
177/177 [==============================] - 0s 268us/step - loss: 0.7366 - accuracy: 0.7006 - val_loss: 1.0478 - val_accuracy: 0.4605
Epoch 3/100
177/177 [==============================] - 0s 247us/step - loss: 0.7420 - accuracy: 0.6893 - val_loss: 1.0510 - val_accuracy: 0.4342
Epoch 4/100
177/177 [==============================] - 0s 220us/step - loss: 0.7314 - accuracy: 0.6949 - val_loss: 1.0562 - val_accuracy: 0.4605
Epoch 5/100
177/177 [==============================] - 0s 212us/step - loss: 0.7337 - accuracy: 0.6836 - val_loss: 1.0586 - val_accuracy: 0.5000
Epoch 6/100
177/177 [==============================] - 0s 207us/step - loss: 0.7294 - accuracy: 0.6949 - val_loss: 1.0608 - val_accuracy: 0.4605
Epoch 7/100
177/177 [==============================] - 0s 229us/step - loss: 0.7331 -

Epoch 57/100
177/177 [==============================] - 0s 303us/step - loss: 0.6763 - accuracy: 0.7232 - val_loss: 1.1217 - val_accuracy: 0.4605
Epoch 58/100
177/177 [==============================] - 0s 336us/step - loss: 0.6722 - accuracy: 0.7232 - val_loss: 1.1168 - val_accuracy: 0.4605
Epoch 59/100
177/177 [==============================] - 0s 232us/step - loss: 0.6616 - accuracy: 0.7345 - val_loss: 1.1098 - val_accuracy: 0.4737
Epoch 60/100
177/177 [==============================] - 0s 156us/step - loss: 0.6602 - accuracy: 0.7458 - val_loss: 1.1103 - val_accuracy: 0.4474
Epoch 61/100
177/177 [==============================] - 0s 158us/step - loss: 0.6515 - accuracy: 0.7401 - val_loss: 1.1021 - val_accuracy: 0.4605
Epoch 62/100
177/177 [==============================] - 0s 155us/step - loss: 0.6597 - accuracy: 0.7119 - val_loss: 1.0984 - val_accuracy: 0.5000
Epoch 63/100
177/177 [==============================] - 0s 161us/step - loss: 0.6561 - accuracy: 0.7232 - val_loss: 1.0924 -

In [209]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel2.history['accuracy'])*100))

train accuracy: 72.69%


In [87]:
df_proba6 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [88]:
df_proba6

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,CFBREBSa104,1,2,1.290039e-09,1.567630e-07,9.999999e-01
1,p0017kpresabs_qual,NRS199,2,2,2.856965e-05,2.843749e-03,9.971277e-01
2,p0017kpresabs_qual,NRS233,1,0,9.999446e-01,4.541289e-06,5.090974e-05
3,p0017kpresabs_qual,SR1746,0,2,6.282367e-02,7.075194e-04,9.364688e-01
4,p0017kpresabs_qual,NRS202,2,2,8.229589e-03,2.163908e-05,9.917488e-01
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,GA27,2,1,3.964591e-03,9.959286e-01,1.068284e-04
604,p0040presabsSTCC_qual,GA231,2,1,3.309226e-04,9.996691e-01,6.232397e-10
605,p0040presabsSTCC_qual,SR1287,0,1,8.445997e-06,9.999915e-01,1.182947e-13
606,p0040presabsSTCC_qual,506,2,0,6.516150e-01,1.480882e-02,3.335762e-01


In [89]:
y_prob6 = df_proba6[df_proba6['phage']=='p0040kpresabs_qual'].iloc[:,-3:]
y_prob6 = y_prob6.to_numpy()
y_prob6

array([[0.05141884, 0.58390534, 0.36467585],
       [0.10108861, 0.3121938 , 0.58671755],
       [0.31517643, 0.4467824 , 0.23804115],
       [0.32365856, 0.43329704, 0.2430444 ],
       [0.3521556 , 0.5906253 , 0.05721916],
       [0.6695274 , 0.14287113, 0.1876014 ],
       [0.16536398, 0.72540855, 0.10922749],
       [0.02954216, 0.58235794, 0.38809988],
       [0.21015587, 0.7127534 , 0.07709076],
       [0.13635716, 0.19357787, 0.6700649 ],
       [0.4946322 , 0.13300906, 0.37235874],
       [0.19735691, 0.5813008 , 0.22134228],
       [0.4946322 , 0.13300906, 0.37235874],
       [0.25047266, 0.4706286 , 0.27889872],
       [0.13635716, 0.19357787, 0.6700649 ],
       [0.17481852, 0.7992746 , 0.0259068 ],
       [0.05083958, 0.3675986 , 0.58156186],
       [0.01369852, 0.70745254, 0.27884892],
       [0.25047266, 0.4706286 , 0.27889872],
       [0.08859741, 0.3068429 , 0.6045597 ],
       [0.33711857, 0.15141867, 0.51146275],
       [0.08859741, 0.3068429 , 0.6045597 ],
       [0.

In [90]:
ovo6 = rocauc_ovo(y_sel_test, y_prob6, average="macro", multi_class="ovo")
ovo6

0.6308749427476158

In [91]:
ovr6 = rocauc_ovr(y_sel_test, y_prob6, average="macro", multi_class="ovr")
ovr6

0.6308749427476158

In [92]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=789,
                                                    stratify=y_sel)

In [93]:
dat7 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat7['test'] = y_sel_test

In [94]:
dat7

,strain,test
120,NRS063,1
187,NRS219,2
96,GA27,2
53,CFBREBSa119,1
197,NRS233,2
...,...,...
47,CFBREBSa110,2
68,CFBRSa05,0
56,CFBREBSa123,0
231,NY360,1


In [95]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [96]:
model_sel3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [110]:
model_sel3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [111]:
model_sel3.fit(X_sel_train, y_sel_train,
          batch_size=16, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 729us/step - loss: 1.1378 - accuracy: 0.3559 - val_loss: 1.0918 - val_accuracy: 0.3289
Epoch 2/100
177/177 [==============================] - 0s 269us/step - loss: 1.0662 - accuracy: 0.4237 - val_loss: 1.0867 - val_accuracy: 0.4079
Epoch 3/100
177/177 [==============================] - 0s 214us/step - loss: 1.0365 - accuracy: 0.4633 - val_loss: 1.0828 - val_accuracy: 0.3816
Epoch 4/100
177/177 [==============================] - 0s 304us/step - loss: 1.0259 - accuracy: 0.4633 - val_loss: 1.0794 - val_accuracy: 0.3947
Epoch 5/100
177/177 [==============================] - 0s 318us/step - loss: 1.0072 - accuracy: 0.5141 - val_loss: 1.0933 - val_accuracy: 0.4079
Epoch 6/100
177/177 [==============================] - 0s 251us/step - loss: 1.0030 - accuracy: 0.5085 - val_loss: 1.0956 - val_accuracy: 0.3947
Epoch 7/100
177/177 [==============================] - 0s 333us/step - loss: 0.9930 -

Epoch 57/100
177/177 [==============================] - 0s 160us/step - loss: 0.8084 - accuracy: 0.6836 - val_loss: 1.0935 - val_accuracy: 0.4737
Epoch 58/100
177/177 [==============================] - 0s 172us/step - loss: 0.8099 - accuracy: 0.6554 - val_loss: 1.0931 - val_accuracy: 0.4211
Epoch 59/100
177/177 [==============================] - 0s 243us/step - loss: 0.8074 - accuracy: 0.6610 - val_loss: 1.0936 - val_accuracy: 0.4605
Epoch 60/100
177/177 [==============================] - 0s 166us/step - loss: 0.8101 - accuracy: 0.6723 - val_loss: 1.0973 - val_accuracy: 0.4474
Epoch 61/100
177/177 [==============================] - 0s 164us/step - loss: 0.8082 - accuracy: 0.6554 - val_loss: 1.0941 - val_accuracy: 0.4342
Epoch 62/100
177/177 [==============================] - 0s 186us/step - loss: 0.8079 - accuracy: 0.6384 - val_loss: 1.1091 - val_accuracy: 0.4737
Epoch 63/100
177/177 [==============================] - 0s 279us/step - loss: 0.8045 - accuracy: 0.6441 - val_loss: 1.1018 -

In [225]:
acc_test_sel3 = model_sel3.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test_sel3*100))

76/76 [==============================] - 0s 90us/step
test accuracy: 50.00%


In [112]:
pred7 = model_sel3.predict_classes(X_sel_test)
pred7

array([1, 2, 1, 0, 1, 0, 0, 2, 2, 2, 0, 2, 2, 2, 1, 2, 1, 0, 1, 1, 1, 2,
       1, 1, 1, 0, 0, 2, 2, 1, 1, 1, 2, 2, 2, 2, 0, 2, 1, 0, 0, 0, 1, 2,
       1, 1, 2, 0, 2, 2, 2, 2, 2, 1, 0, 1, 2, 1, 1, 2, 1, 0, 2, 1, 2, 1,
       1, 1, 0, 1, 1, 2, 1, 1, 2, 1])

In [113]:
dat7['pred'] = pred7
dat7

,strain,test,pred
120,NRS063,1,1
187,NRS219,2,2
96,GA27,2,1
53,CFBREBSa119,1,0
197,NRS233,2,1
...,...,...,...
47,CFBREBSa110,2,2
68,CFBRSa05,0,1
56,CFBREBSa123,0,1
231,NY360,1,2


In [114]:
proba7 = model_sel3.predict_proba(X_sel_test)
dat_proba7 = pd.DataFrame(proba7)

In [115]:
dat_proba7

,0,1,2
0,0.117554,0.845996,0.036450
1,0.042698,0.234220,0.723082
2,0.383970,0.479381,0.136649
3,0.545839,0.193339,0.260822
4,0.161795,0.549704,0.288501
...,...,...,...
71,0.179684,0.248874,0.571441
72,0.368936,0.500525,0.130539
73,0.136278,0.512505,0.351217
74,0.179684,0.248874,0.571441


In [116]:
dat_proba7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba7.csv", index = False,
         header=None)

In [117]:
dat7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/7p40.csv", index = False,
         header=None)

In [229]:
hist_sel3 = model_sel3.fit(X_sel_train, y_sel_train,
          batch_size=16, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 232us/step - loss: 0.7398 - accuracy: 0.7006 - val_loss: 1.0866 - val_accuracy: 0.4868
Epoch 2/100
177/177 [==============================] - 0s 307us/step - loss: 0.7414 - accuracy: 0.7119 - val_loss: 1.0821 - val_accuracy: 0.5132
Epoch 3/100
177/177 [==============================] - 0s 220us/step - loss: 0.7401 - accuracy: 0.7175 - val_loss: 1.0845 - val_accuracy: 0.5132
Epoch 4/100
177/177 [==============================] - 0s 200us/step - loss: 0.7348 - accuracy: 0.7288 - val_loss: 1.0740 - val_accuracy: 0.4737
Epoch 5/100
177/177 [==============================] - 0s 220us/step - loss: 0.7524 - accuracy: 0.6780 - val_loss: 1.0751 - val_accuracy: 0.4211
Epoch 6/100
177/177 [==============================] - 0s 237us/step - loss: 0.7442 - accuracy: 0.7006 - val_loss: 1.0671 - val_accuracy: 0.4737
Epoch 7/100
177/177 [==============================] - 0s 228us/step - loss: 0.7419 -

Epoch 57/100
177/177 [==============================] - 0s 207us/step - loss: 0.6729 - accuracy: 0.7119 - val_loss: 1.1517 - val_accuracy: 0.4474
Epoch 58/100
177/177 [==============================] - 0s 223us/step - loss: 0.6785 - accuracy: 0.7062 - val_loss: 1.1511 - val_accuracy: 0.4474
Epoch 59/100
177/177 [==============================] - 0s 927us/step - loss: 0.6704 - accuracy: 0.7232 - val_loss: 1.1535 - val_accuracy: 0.4211
Epoch 60/100
177/177 [==============================] - 0s 266us/step - loss: 0.6654 - accuracy: 0.7345 - val_loss: 1.1538 - val_accuracy: 0.4342
Epoch 61/100
177/177 [==============================] - 0s 699us/step - loss: 0.6762 - accuracy: 0.7232 - val_loss: 1.1637 - val_accuracy: 0.4605
Epoch 62/100
177/177 [==============================] - 0s 264us/step - loss: 0.6715 - accuracy: 0.7175 - val_loss: 1.1574 - val_accuracy: 0.4342
Epoch 63/100
177/177 [==============================] - 0s 232us/step - loss: 0.6636 - accuracy: 0.7175 - val_loss: 1.1586 -

In [230]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel3.history['accuracy'])*100))

train accuracy: 72.11%


In [97]:
df_proba7 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [98]:
df_proba7

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS271,1,0,0.854929,0.144940,0.000130
1,p0017kpresabs_qual,SR2852,2,2,0.000001,0.000075,0.999923
2,p0017kpresabs_qual,CA39,2,0,0.959992,0.013406,0.026602
3,p0017kpresabs_qual,NRS266,1,2,0.000018,0.000003,0.999980
4,p0017kpresabs_qual,NY356,2,2,0.009156,0.411991,0.578852
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,CFBREBSa110,2,2,0.044135,0.010244,0.945621
604,p0040presabsSTCC_qual,CFBRSa05,0,2,0.145032,0.408530,0.446439
605,p0040presabsSTCC_qual,CFBREBSa123,0,0,0.994623,0.004152,0.001224
606,p0040presabsSTCC_qual,NY360,1,1,0.000795,0.948566,0.050639


In [99]:
y_prob7 = df_proba7[df_proba7['phage']=='p0040kpresabs_qual'].iloc[:,-3:]
y_prob7 = y_prob7.to_numpy()
y_prob7

array([[0.11755378, 0.8459965 , 0.03644972],
       [0.04269802, 0.23422003, 0.72308195],
       [0.3839702 , 0.4793813 , 0.13664854],
       [0.5458387 , 0.19333918, 0.26082212],
       [0.1617952 , 0.54970366, 0.2885011 ],
       [0.40070727, 0.33775336, 0.2615393 ],
       [0.47838765, 0.3219237 , 0.19968863],
       [0.07698642, 0.32936263, 0.59365094],
       [0.17968413, 0.24887443, 0.5714414 ],
       [0.17968413, 0.24887443, 0.5714414 ],
       [0.5694695 , 0.36324418, 0.06728628],
       [0.00356365, 0.09140137, 0.90503496],
       [0.04269802, 0.23422003, 0.72308195],
       [0.00356365, 0.09140137, 0.90503496],
       [0.00779021, 0.8835956 , 0.10861427],
       [0.25838095, 0.33845374, 0.40316525],
       [0.2820515 , 0.6775162 , 0.04043233],
       [0.6649023 , 0.21456218, 0.12053547],
       [0.2362025 , 0.690679  , 0.0731185 ],
       [0.22566627, 0.55320513, 0.22112855],
       [0.1617952 , 0.54970366, 0.2885011 ],
       [0.01930976, 0.15389492, 0.8267954 ],
       [0.

In [100]:
ovo7 = rocauc_ovo(y_sel_test, y_prob7, average="macro", multi_class="ovo")
ovo7

0.645114281462899

In [101]:
ovr7 = rocauc_ovr(y_sel_test, y_prob7, average="macro", multi_class="ovr")
ovr7

0.645114281462899

In [102]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=890,
                                                    stratify=y_sel)

In [103]:
dat8 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat8['test'] = y_sel_test

In [104]:
dat8

,strain,test
130,NRS103,2
92,EUH25,0
143,NRS148,2
107,NRS001,1
166,NRS192,0
...,...,...
139,NRS113,1
24,BCH-SA-09,2
133,NRS106,2
62,CFBREBSa131,2


In [105]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [106]:
model_sel4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [123]:
model_sel4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [124]:
model_sel4.fit(X_sel_train, y_sel_train,
          batch_size=16, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 673us/step - loss: 1.0916 - accuracy: 0.4576 - val_loss: 1.0525 - val_accuracy: 0.4474
Epoch 2/100
177/177 [==============================] - 0s 228us/step - loss: 1.0564 - accuracy: 0.4633 - val_loss: 1.0568 - val_accuracy: 0.4474
Epoch 3/100
177/177 [==============================] - 0s 233us/step - loss: 1.0291 - accuracy: 0.4802 - val_loss: 1.0588 - val_accuracy: 0.4342
Epoch 4/100
177/177 [==============================] - 0s 285us/step - loss: 1.0173 - accuracy: 0.4746 - val_loss: 1.0593 - val_accuracy: 0.4605
Epoch 5/100
177/177 [==============================] - 0s 237us/step - loss: 1.0145 - accuracy: 0.5141 - val_loss: 1.0632 - val_accuracy: 0.4605
Epoch 6/100
177/177 [==============================] - 0s 304us/step - loss: 0.9982 - accuracy: 0.4972 - val_loss: 1.0529 - val_accuracy: 0.4079
Epoch 7/100
177/177 [==============================] - 0s 254us/step - loss: 0.9926 -

In [246]:
acc_test_sel4 = model_sel4.evaluate(X_sel_test, y_sel_test)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_sel4*100))

76/76 [==============================] - 0s 63us/step
over-sampling test accuracy: 52.63%


In [125]:
pred8 = model_sel4.predict_classes(X_sel_test)
pred8

array([2, 0, 2, 0, 1, 1, 1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 0, 1, 0,
       0, 0, 1, 1, 0, 2, 2, 2, 2, 1, 1, 1, 2, 2, 0, 2, 1, 2, 1, 1, 2, 0,
       2, 0, 2, 0, 0, 2, 1, 1, 1, 0, 0, 1, 2, 1, 1, 0, 1, 2, 0, 2, 0, 2,
       2, 2, 1, 1, 0, 1, 1, 0, 0, 0])

In [126]:
dat8['pred'] = pred8
dat8

,strain,test,pred
130,NRS103,2,2
92,EUH25,0,0
143,NRS148,2,2
107,NRS001,1,0
166,NRS192,0,1
...,...,...,...
139,NRS113,1,1
24,BCH-SA-09,2,1
133,NRS106,2,0
62,CFBREBSa131,2,0


In [127]:
proba8 = model_sel4.predict_proba(X_sel_test)
dat_proba8 = pd.DataFrame(proba8)

In [128]:
dat_proba8

,0,1,2
0,0.065021,0.425737,0.509242
1,0.353188,0.303337,0.343475
2,0.051844,0.473591,0.474565
3,0.574330,0.350626,0.075044
4,0.216782,0.485269,0.297949
...,...,...,...
71,0.060622,0.515135,0.424242
72,0.172904,0.555872,0.271225
73,0.489310,0.363842,0.146848
74,0.489310,0.363842,0.146848


In [129]:
dat_proba8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba8.csv", index = False,
         header=None)

In [130]:
dat8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/8p40.csv", index = False,
         header=None)

In [250]:
hist_sel4 = model_sel4.fit(X_sel_train, y_sel_train,
          batch_size=16, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 216us/step - loss: 0.7649 - accuracy: 0.6610 - val_loss: 1.0274 - val_accuracy: 0.5395
Epoch 2/100
177/177 [==============================] - 0s 241us/step - loss: 0.7560 - accuracy: 0.6780 - val_loss: 1.0192 - val_accuracy: 0.5395
Epoch 3/100
177/177 [==============================] - 0s 211us/step - loss: 0.7516 - accuracy: 0.6949 - val_loss: 1.0186 - val_accuracy: 0.5263
Epoch 4/100
177/177 [==============================] - 0s 194us/step - loss: 0.7570 - accuracy: 0.6780 - val_loss: 1.0229 - val_accuracy: 0.5263
Epoch 5/100
177/177 [==============================] - 0s 198us/step - loss: 0.7562 - accuracy: 0.6554 - val_loss: 1.0216 - val_accuracy: 0.5526
Epoch 6/100
177/177 [==============================] - 0s 193us/step - loss: 0.7603 - accuracy: 0.6723 - val_loss: 1.0138 - val_accuracy: 0.5658
Epoch 7/100
177/177 [==============================] - 0s 176us/step - loss: 0.7540 -

In [251]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel4.history['accuracy'])*100))

train accuracy: 70.68%


In [107]:
df_proba8 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [108]:
df_proba8

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS104,0,1,8.210638e-02,0.917809,8.477923e-05
1,p0017kpresabs_qual,NRS071,0,2,1.015229e-02,0.255327,7.345203e-01
2,p0017kpresabs_qual,NRS072,1,2,3.642946e-03,0.000002,9.963547e-01
3,p0017kpresabs_qual,BCH-SA-12,0,2,2.809318e-03,0.099188,8.980029e-01
4,p0017kpresabs_qual,CFBREBSa125,2,0,4.875667e-01,0.088657,4.237761e-01
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS113,1,2,2.686909e-07,0.093121,9.068785e-01
604,p0040presabsSTCC_qual,BCH-SA-09,2,1,5.529492e-07,0.999990,9.559324e-06
605,p0040presabsSTCC_qual,NRS106,2,1,1.115902e-03,0.998816,6.852559e-05
606,p0040presabsSTCC_qual,CFBREBSa131,2,2,2.708040e-07,0.156768,8.432316e-01


In [109]:
y_prob8 = df_proba8[df_proba8['phage']=='p0040kpresabs_qual'].iloc[:,-3:]
y_prob8 = y_prob8.to_numpy()
y_prob8

array([[0.06502149, 0.42573664, 0.5092419 ],
       [0.35318816, 0.30333686, 0.343475  ],
       [0.05184364, 0.47359097, 0.47456536],
       [0.57433015, 0.35062632, 0.07504351],
       [0.21678227, 0.48526892, 0.29794878],
       [0.14364265, 0.49772835, 0.358629  ],
       [0.16133824, 0.67639565, 0.16226612],
       [0.7120913 , 0.22905402, 0.05885464],
       [0.06190963, 0.50238204, 0.43570828],
       [0.10827925, 0.36898807, 0.52273273],
       [0.07288651, 0.77674985, 0.15036368],
       [0.30956638, 0.5371874 , 0.1532462 ],
       [0.09097935, 0.54213053, 0.36689007],
       [0.1671836 , 0.557644  , 0.27517232],
       [0.07288651, 0.77674985, 0.15036368],
       [0.18535659, 0.6803928 , 0.13425054],
       [0.09204357, 0.46087155, 0.44708487],
       [0.10827925, 0.36898807, 0.52273273],
       [0.03084035, 0.38422337, 0.5849363 ],
       [0.6035241 , 0.254427  , 0.14204891],
       [0.11814371, 0.47777766, 0.4040787 ],
       [0.48949414, 0.36511445, 0.14539143],
       [0.

In [110]:
ovo8 = rocauc_ovo(y_sel_test, y_prob8, average="macro", multi_class="ovo")
ovo8

0.6692453209301393

In [111]:
ovr8 = rocauc_ovr(y_sel_test, y_prob8, average="macro", multi_class="ovr")
ovr8

0.6692453209301393

In [112]:
ovos2 = [ovo5, ovo6, ovo7, ovo8]
np.mean(ovos2)

0.6327603133508721

In [113]:
np.std(ovos2)

0.030380799690129925

In [114]:
ovrs2 = [ovr5, ovr6, ovr7, ovr8]
np.mean(ovrs2)

0.6327603133508721

In [115]:
np.std(ovrs2)

0.030380799690129925

In [252]:
accs_l= [acc_test_sel, acc_test_sel2, acc_test_sel3, acc_test_sel4]

In [253]:
mean_l = np.mean(accs_l)
print('test accuracy mean after lasso: %.2f%%' % (mean_l*100))

test accuracy mean after lasso: 48.68%


In [254]:
std_l = np.std(accs_l)
print('test accuracy standard deviation after lasso:', std_l)

test accuracy standard deviation after lasso: 0.03947368760903739


In [255]:
accs_train_l = [np.mean(hist_sel.history['accuracy']), np.mean(hist_sel2.history['accuracy']), np.mean(hist_sel3.history['accuracy']),
             np.mean(hist_sel4.history['accuracy'])]

In [256]:
mean_train_l = np.mean(accs_train_l)
print('train accuracy mean after lasso: %.2f%%' % (mean_train_l*100))

train accuracy mean after lasso: 73.31%


In [257]:
std_train_l = np.std(accs_train_l)
print('train accuracy standard deviation after lasso:', std_train_l)

train accuracy standard deviation after lasso: 0.026665628
